In [1]:
import dask.dataframe as dd


In [2]:
df = dd.read_parquet("data/tracks_4326.parquet")


In [3]:
for x in  df.columns:
    print(x)


region
sc_orient
track
segment_dist
solar_elevation
segment_id
background_rate
cycle
pair
rgt
snowcover
yapc_score
height
y_atc
landcover
x_atc
relief
quality_ph
atl03_cnf
atl08_class
spot
time
geoid_height
orthometric_height
h_alos_dem
delta_alos_dem
abs_delta_alos_dem
h_aster_dem
delta_aster_dem
abs_delta_aster_dem
h_copernicus_dеm
delta_copernicus_dеm
abs_delta_copernicus_dеm
h_fab_dem
delta_fab_dem
abs_delta_fab_dem
h_nasa_dem
delta_nasa_dem
abs_delta_nasa_dem
h_srtm_dem
delta_srtm_dem
abs_delta_srtm_dem
h_tan_dem
delta_tan_dem
abs_delta_tan_dem
year
lulc_class
lulc_name
alos_dem_slope
aster_dem_slope
copernicus_dеm_slope
fab_dem_slope
nasa_dem_slope
srtm_dem_slope
tan_dem_slope
alos_dem_geomorphon
alos_dem_landform
aster_dem_geomorphon
aster_dem_landform
copernicus_dеm_geomorphon
copernicus_dеm_landform
fab_dem_geomorphon
fab_dem_landform
nasa_dem_geomorphon
nasa_dem_landform
srtm_dem_geomorphon
srtm_dem_landform
tan_dem_geomorphon
tan_dem_landform
alos_dem_2000
aster_dem_2000
cop

In [4]:
print(df.head())
print(df.describe())

   region  sc_orient  track  segment_dist  solar_elevation  segment_id  \
0     6.0        1.0    1.0  1.472979e+07       -40.574768    735401.0   
1     6.0        1.0    1.0  1.472979e+07       -40.574768    735401.0   
2     6.0        1.0    1.0  1.472979e+07       -40.574768    735401.0   
3     6.0        1.0    1.0  1.472979e+07       -40.574768    735401.0   
4     6.0        1.0    1.0  1.472979e+07       -40.574768    735401.0   

   background_rate  cycle  pair    rgt  ...  alos_dem_twi  aster_dem_twi  \
0      2756.054548    1.0   0.0  556.0  ...      5.623988       4.552485   
1      2756.054548    1.0   0.0  556.0  ...      5.623988       4.552485   
2      2756.054548    1.0   0.0  556.0  ...      5.623988       4.552485   
3      2756.054548    1.0   0.0  556.0  ...      4.273866       4.552485   
4      2756.054548    1.0   0.0  556.0  ...      4.273866       4.904620   

   copernicus_dеm_twi  fab_dem_twi  nasa_dem_twi  srtm_dem_twi  tan_dem_twi  \
0            5.0888

In [2]:
import duckdb


In [5]:
duckdb.sql("""
    CREATE OR REPLACE TABLE temp_table AS
    SELECT
        *,
        "copernicus_dеm_slope" AS "copernicus_dem_slope"
    FROM data/tracks_3857_no_geometry.parquet
""")

duckdb.sql("""
    ALTER TABLE temp_table DROP COLUMN "copernicus_dеm_slope"
""")

duckdb.sql("""
    COPY temp_table TO 'data/tracks_no_geometry_fixed.parquet' (FORMAT PARQUET)
""")


ParserException: Parser Error: syntax error at or near "/"

In [9]:
import duckdb
duckdb.sql("""
COPY (
  SELECT * EXCLUDE geometry
  FROM 'data/tracks_3857.parquet'
)
TO 'data/tracks_3857_no_geometry.parquet' (FORMAT 'parquet');
""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [1]:
import duckdb
duckdb.sql("DESCRIBE SELECT * FROM 'data/tracks_no_geometry_fixed.parquet'").show()



InvalidInputException: Invalid Input Error: No magic bytes found at end of file 'data/tracks_no_geometry_fixed.parquet'

In [ ]:
import duckdb

duckdb.sql("""
    COPY (
        SELECT * EXCLUDE geometry FROM 'data/icesat2_dem.parquet'
        WHERE atl03_cnf = 4 AND atl08_class = 1
    )
    TO 'data/icesat2_dem_filtered.parquet' (FORMAT PARQUET)
""")


In [ ]:
import duckdb

# 1. Відкриваємо Parquet як DuckDB-таблицю (заміни шлях на лапки!)
duckdb.sql("""
    CREATE OR REPLACE TABLE temp_table AS
    SELECT
        *,
        "copernicus_dеm_slope" AS "copernicus_dem_slope"
    FROM 'data/tracks_3857_no_geometry.parquet'
""")

# 2. Видаляємо стару колонку з кирилицею (опціонально)
duckdb.sql("""
    ALTER TABLE temp_table DROP COLUMN "copernicus_dеm_slope"
""")

# 3. Зберігаємо як новий Parquet-файл з правильними назвами колонок
duckdb.sql("""
    COPY temp_table TO 'data/tracks_no_geometry_fixed.parquet' (FORMAT PARQUET)
""")



FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [1]:
import duckdb

track = 1.0
rgt = 396.0
spot = 1.0
date = '2019-07-24'
hand_col = 'alos_dem_2000'
dem = 'alos_dem'

sql = f"""
SELECT x, y, delta_{dem}, abs_delta_{dem}
FROM 'data/tracks_3857_1.parquet'
WHERE track={track} AND rgt={rgt} AND spot={spot}
  AND DATE(time) = '{date}'
  AND atl03_cnf = 4 AND atl08_class = 1
  AND {hand_col} BETWEEN 0 AND 5
"""

df = duckdb.query(sql).to_df()
print(df.shape)
print(df.head())


(130, 4)
           x          y  delta_alos_dem  abs_delta_alos_dem
0  24.976054  47.730581       -0.166000            0.166000
1  24.976054  47.730581       -0.166000            0.166000
2  24.976051  47.730600        1.131974            1.131974
3  24.976051  47.730600        1.131974            1.131974
4  24.976051  47.730607        0.872940            0.872940


In [4]:
import duckdb
duckdb.query("""
  SELECT *
  FROM 'data/tracks_3857_1.parquet'
  WHERE track=1.0 AND rgt=396.0 AND spot=1.0
    AND DATE(time) = '2019-07-24'
    AND delta_alos_dem IS NOT NULL AND h_alos_dem IS NOT NULL
    AND atl03_cnf = 4 AND atl08_class = 1
    AND alos_dem_2000 BETWEEN 0 AND 5
  ORDER BY x
""").to_df()
print(duckdb.query("""
  SELECT *
  FROM 'data/tracks_3857_1.parquet'
  WHERE track=1.0 AND rgt=396.0 AND spot=1.0
    AND DATE(time) = '2019-07-24'
    AND delta_alos_dem IS NOT NULL AND h_alos_dem IS NOT NULL
    AND atl03_cnf = 4 AND atl08_class = 1
    AND alos_dem_2000 BETWEEN 0 AND 5
  ORDER BY x
""").to_df())

     region  sc_orient  track  segment_dist  solar_elevation  segment_id  \
0       2.0        0.0    1.0  5.314929e+06       -17.953997    265053.0   
1       2.0        0.0    1.0  5.314949e+06       -17.953997    265054.0   
2       2.0        0.0    1.0  5.314949e+06       -17.953997    265054.0   
3       2.0        0.0    1.0  5.314929e+06       -17.953997    265053.0   
4       2.0        0.0    1.0  5.314949e+06       -17.953997    265054.0   
..      ...        ...    ...           ...              ...         ...   
125     2.0        0.0    1.0  5.313587e+06       -17.964470    264986.0   
126     2.0        0.0    1.0  5.313587e+06       -17.964470    264986.0   
127     2.0        0.0    1.0  5.313567e+06       -17.964621    264985.0   
128     2.0        0.0    1.0  5.313587e+06       -17.964470    264986.0   
129     2.0        0.0    1.0  5.313567e+06       -17.964621    264985.0   

     background_rate  cycle  pair    rgt  ...  fab_dem_twi  nasa_dem_twi  \
0       113

In [6]:
# Тимчасово відключити фільтр hand_range:
sql = f"""
    SELECT x, y, delta_{dem}, abs_delta_{dem}
    FROM 'data/tracks_3857_1.parquet'
    WHERE track={track} AND rgt={rgt} AND spot={spot}
        AND DATE(time) = '{date}'
        AND atl03_cnf = 4 AND atl08_class = 1
"""

print(duckdb.query(sql).to_df())

              x          y  delta_alos_dem  abs_delta_alos_dem
0     24.976597  47.726559       10.839737           10.839737
1     24.976593  47.726585       10.836197           10.836197
2     24.976593  47.726585       10.691544           10.691544
3     24.976593  47.726585       11.201798           11.201798
4     24.976591  47.726604       11.672135           11.672135
...         ...        ...             ...                 ...
1838  24.927032  48.071453        1.736813            1.736813
1839  24.927030  48.071472        1.805600            1.805600
1840  24.927030  48.071472        2.255673            2.255673
1841  24.927029  48.071479        3.567990            3.567990
1842  24.927020  48.071542        3.049008            3.049008

[1843 rows x 4 columns]


In [3]:
import pandas as pd
df = duckdb.query("SELECT alos_dem_2000 FROM 'data/tracks_3857_1.parquet'").to_df()
print(df['alos_dem_2000'].isna().sum(), df['alos_dem_2000'].dtype)
print(df['alos_dem_2000'].sample(10))


765194 float64
6773667    237.74
6967909    242.00
4638358    291.00
7071200    138.00
633895      40.00
2265507    249.00
3987542    215.00
2621187    261.00
1206081    323.14
4181338    198.00
Name: alos_dem_2000, dtype: float64


In [ ]:
import pandas as pd

# 1. Завантажуємо Parquet
df = pd.read_parquet("data/tracks_3857_no_geometry.parquet")

# 2. Замінюємо всі кириличні “е” на латиничні “e” в усіх назвах колонок
df.columns = [col.replace("е", "e") for col in df.columns]

# 3. Зберігаємо новий файл
df.to_parquet("data/tracks_3857_1.parquet")


In [1]:
import pandas as pd
import dask.dataframe as dd

df = dd.read_parquet("data/tracks_3857_1.parquet")

df.columns

Index(['region', 'sc_orient', 'track', 'segment_dist', 'solar_elevation',
       'segment_id', 'background_rate', 'cycle', 'pair', 'rgt', 'snowcover',
       'yapc_score', 'height', 'y_atc', 'landcover', 'x_atc', 'relief',
       'quality_ph', 'atl03_cnf', 'atl08_class', 'spot', 'time',
       'geoid_height', 'orthometric_height', 'h_alos_dem', 'delta_alos_dem',
       'abs_delta_alos_dem', 'h_aster_dem', 'delta_aster_dem',
       'abs_delta_aster_dem', 'h_copernicus_dem', 'delta_copernicus_dem',
       'abs_delta_copernicus_dem', 'h_fab_dem', 'delta_fab_dem',
       'abs_delta_fab_dem', 'h_nasa_dem', 'delta_nasa_dem',
       'abs_delta_nasa_dem', 'h_srtm_dem', 'delta_srtm_dem',
       'abs_delta_srtm_dem', 'h_tan_dem', 'delta_tan_dem', 'abs_delta_tan_dem',
       'year', 'lulc_class', 'lulc_name', 'alos_dem_slope', 'aster_dem_slope',
       'copernicus_dem_slope', 'fab_dem_slope', 'nasa_dem_slope',
       'srtm_dem_slope', 'tan_dem_slope', 'alos_dem_geomorphon',
       'alos_dem_landf

In [3]:
import pandas as pd

df_new = pd.read_parquet("data/tracks_3857_1.parquet")
print([col for col in df_new.columns if "dem" in col])

['h_alos_dem', 'delta_alos_dem', 'abs_delta_alos_dem', 'h_aster_dem', 'delta_aster_dem', 'abs_delta_aster_dem', 'h_copernicus_dem', 'delta_copernicus_dem', 'abs_delta_copernicus_dem', 'h_fab_dem', 'delta_fab_dem', 'abs_delta_fab_dem', 'h_nasa_dem', 'delta_nasa_dem', 'abs_delta_nasa_dem', 'h_srtm_dem', 'delta_srtm_dem', 'abs_delta_srtm_dem', 'h_tan_dem', 'delta_tan_dem', 'abs_delta_tan_dem', 'alos_dem_slope', 'aster_dem_slope', 'copernicus_dem_slope', 'fab_dem_slope', 'nasa_dem_slope', 'srtm_dem_slope', 'tan_dem_slope', 'alos_dem_geomorphon', 'alos_dem_landform', 'aster_dem_geomorphon', 'aster_dem_landform', 'copernicus_dem_geomorphon', 'copernicus_dem_landform', 'fab_dem_geomorphon', 'fab_dem_landform', 'nasa_dem_geomorphon', 'nasa_dem_landform', 'srtm_dem_geomorphon', 'srtm_dem_landform', 'tan_dem_geomorphon', 'tan_dem_landform', 'alos_dem_2000', 'aster_dem_2000', 'copernicus_dem_2000', 'fab_dem_2000', 'nasa_dem_2000', 'srtm_dem_2000', 'tan_dem_2000', 'alos_dem_stream', 'aster_dem_str

In [1]:
import duckdb
con = duckdb.connect()
print(con.execute("SELECT DISTINCT lulc_class FROM 'data/tracks_3857_1.parquet' WHERE delta_tan_dem IS NOT NULL AND atl03_cnf = 4 AND atl08_class = 1").fetchall())
con.close()


[(None,), (2.0,), (1.0,), (11.0,), (7.0,), (5.0,)]


In [2]:
import pandas as pd
df = pd.read_parquet("data/tracks_3857_1.parquet")
print(df["lulc_name"].unique())
print(df["tan_dem_landform"].unique())

['Trees' 'Rangeland' None 'Built Area' 'Crops' 'Water' 'Bare Ground']
['Slope' 'Spur' 'Hollow' 'Valley' 'Ridge' None 'Shoulder' 'Peak'
 'Footslope' 'Pit']


In [ ]:
for x in  df.columns:
    print(x)


In [2]:
print(df.head())
print(df.describe())

   region  sc_orient  track  segment_dist  solar_elevation  segment_id  \
0     6.0        1.0    1.0  1.472979e+07       -40.574768    735401.0   
1     6.0        1.0    1.0  1.472979e+07       -40.574768    735401.0   
2     6.0        1.0    1.0  1.472979e+07       -40.574768    735401.0   
3     6.0        1.0    1.0  1.472979e+07       -40.574768    735401.0   
4     6.0        1.0    1.0  1.472979e+07       -40.574768    735401.0   

   background_rate  cycle  pair    rgt  ...  fab_dem_twi  nasa_dem_twi  \
0      2756.054548    1.0   0.0  556.0  ...     5.662695      4.363451   
1      2756.054548    1.0   0.0  556.0  ...     5.662695      4.363451   
2      2756.054548    1.0   0.0  556.0  ...     5.662695      4.363451   
3      2756.054548    1.0   0.0  556.0  ...     5.662695      4.363451   
4      2756.054548    1.0   0.0  556.0  ...     5.535255      4.164953   

   srtm_dem_twi  tan_dem_twi       __index_level_0__  \
0      4.352303     5.088826 2018-11-04 01:05:31.626  

In [8]:
print(df['pair'].unique().compute())


0    0.0
0    1.0
Name: pair, dtype: float64


In [29]:
import duckdb

# Замініть на реальний шлях до твого parquet-файлу
parquet_path = "data/tracks_3857_1.parquet"
con = duckdb.connect()

# Вивести всі унікальні класи LULC та їх кількість
df = con.execute(f"""
  SELECT
    COUNT(*) AS total,
    COUNT(lulc_class) AS not_null,
    COUNT(*) - COUNT(lulc_class) AS null_count
FROM 'data/tracks_3857_1.parquet'
WHERE
    atl03_cnf = 4
    AND atl08_class = 1;


""").fetchdf()

print(df)
# con.close()


     total  not_null  null_count
0  2022807   1982195       40612


In [5]:
dem_list = [
    "alos_dem", "aster_dem", "copernicus_dem", "fab_dem",
    "nasa_dem", "srtm_dem", "tan_dem"
]
for dem in dem_list:
    sql = f"""
        SELECT COUNT(*)
        FROM '{parquet_path}'
        WHERE lulc_name = 'Bare Ground' AND delta_{dem} IS NOT NULL
    """
    count = con.execute(sql).fetchone()[0]
    print(f"{dem}: {count}")


alos_dem: 12
aster_dem: 12
copernicus_dem: 12
fab_dem: 12
nasa_dem: 12
srtm_dem: 12
tan_dem: 12


In [12]:
df = con.execute("""
  SELECT *
FROM 'data/tracks_3857_1.parquet'
WHERE lulc_name = 'Bare Ground'
  AND atl03_cnf = 4
  AND atl08_class = 1
""").fetchdf()
print(df)

Empty DataFrame
Columns: [region, sc_orient, track, segment_dist, solar_elevation, segment_id, background_rate, cycle, pair, rgt, snowcover, yapc_score, height, y_atc, landcover, x_atc, relief, quality_ph, atl03_cnf, atl08_class, spot, time, geoid_height, orthometric_height, h_alos_dem, delta_alos_dem, abs_delta_alos_dem, h_aster_dem, delta_aster_dem, abs_delta_aster_dem, h_copernicus_dem, delta_copernicus_dem, abs_delta_copernicus_dem, h_fab_dem, delta_fab_dem, abs_delta_fab_dem, h_nasa_dem, delta_nasa_dem, abs_delta_nasa_dem, h_srtm_dem, delta_srtm_dem, abs_delta_srtm_dem, h_tan_dem, delta_tan_dem, abs_delta_tan_dem, year, lulc_class, lulc_name, alos_dem_slope, aster_dem_slope, copernicus_dem_slope, fab_dem_slope, nasa_dem_slope, srtm_dem_slope, tan_dem_slope, alos_dem_geomorphon, alos_dem_landform, aster_dem_geomorphon, aster_dem_landform, copernicus_dem_geomorphon, copernicus_dem_landform, fab_dem_geomorphon, fab_dem_landform, nasa_dem_geomorphon, nasa_dem_landform, srtm_dem_geomor

In [9]:
df = con.execute("""
  SELECT
    MIN(alos_dem_slope) as alos_min,
    MAX(alos_dem_slope) as alos_max,
    AVG(alos_dem_slope) as alos_avg,
    MIN(aster_dem_slope) as aster_min,
    MAX(aster_dem_slope) as aster_max,
    AVG(aster_dem_slope) as aster_avg,
    MIN(copernicus_dem_slope) as copernicus_min,
    MAX(copernicus_dem_slope) as copernicus_max,
    AVG(copernicus_dem_slope) as copernicus_avg,
    MIN(fab_dem_slope) as fab_min,
    MAX(fab_dem_slope) as fab_max,
    AVG(fab_dem_slope) as fab_avg,
    MIN(nasa_dem_slope) as nasa_min,
    MAX(nasa_dem_slope) as nasa_max,
    AVG(nasa_dem_slope) as nasa_avg,
    MIN(srtm_dem_slope) as srtm_min,
    MAX(srtm_dem_slope) as srtm_max,
    AVG(srtm_dem_slope) as srtm_avg,
    MIN(tan_dem_slope) as tan_min,
    MAX(tan_dem_slope) as tan_max,
    AVG(tan_dem_slope) as tan_avg
FROM 'data/tracks_3857_1.parquet';
""").fetchdf()
print(df.T)

                        0
alos_min         0.380607
alos_max        56.167126
alos_avg        17.812838
aster_min        0.380610
aster_max       56.472008
aster_avg       17.497994
copernicus_min   0.029752
copernicus_max  48.647015
copernicus_avg  18.121265
fab_min          0.020253
fab_max         48.572330
fab_avg         17.798044
nasa_min         0.094691
nasa_max        50.299198
nasa_avg        17.109656
srtm_min         0.380610
srtm_max        50.179173
srtm_avg        17.269574
tan_min          0.029657
tan_max         48.647060
tan_avg         18.134719


In [11]:
df = con.execute("""
SELECT
  COUNT(DISTINCT alos_dem_slope) as alos_unique,
  COUNT(DISTINCT aster_dem_slope) as aster_unique,
  COUNT(DISTINCT copernicus_dem_slope) as copernicus_unique,
  COUNT(DISTINCT fab_dem_slope) as fab_unique,
  COUNT(DISTINCT nasa_dem_slope) as nasa_unique,
  COUNT(DISTINCT srtm_dem_slope) as srtm_unique,
  COUNT(DISTINCT tan_dem_slope) as tan_unique
FROM 'data/tracks_3857_1.parquet'
""").fetchdf()
print(df.T)



                       0
alos_unique         3978
aster_unique        4250
copernicus_unique  61875
fab_unique         61884
nasa_unique        61889
srtm_unique         3538
tan_unique         61872


In [12]:
import duckdb


In [14]:
def get_icesat2_tracks(
    year=None,
    dem=None,
    max_abs_error=None,
    track=None,
    rgt=None,
    cycle=None,
    spot=None,
    y_atc_min=None,
    y_atc_max=None
):
    sql = "SELECT x, y, track, year, rgt, cycle, spot"
    if dem:
        sql += f", delta_{dem}, abs_delta_{dem}"
    sql += " FROM 'data/tracks_3857_1.parquet' WHERE 1=1"
    if year:
        sql += f" AND year = {year}"
    if track is not None:
        sql += f" AND track = {track}"
    if rgt is not None:
        sql += f" AND rgt = {rgt}"
    if cycle is not None:
        sql += f" AND cycle = {cycle}"
    if spot is not None:
        sql += f" AND spot = {spot}"
    if y_atc_min is not None and y_atc_max is not None:
        sql += f" AND y_atc BETWEEN {y_atc_min} AND {y_atc_max}"
    if dem and max_abs_error is not None:
        sql += f" AND abs_delta_{dem} <= {max_abs_error}"
    print("SQL-квері:\n", sql)
    df = duckdb.query(sql).to_df()
    # (далі формуєш geojson...)
    return df  # або твій geojson

In [20]:
df = get_icesat2_tracks(
    year=2020,
    dem='fab_dem',
    track=1.0,
    rgt=998,

)
print(df)

SQL-квері:
 SELECT x, y, track, year, rgt, cycle, spot, delta_fab_dem, abs_delta_fab_dem FROM 'data/tracks_3857_1.parquet' WHERE 1=1 AND year = 2020 AND track = 1.0 AND rgt = 998 AND abs_delta_fab_dem <= 2.5
               x          y  track  year    rgt  cycle  spot  delta_fab_dem  \
0      24.998010  48.119004    1.0  2020  998.0    8.0   1.0      -1.403421   
1      24.998010  48.119004    1.0  2020  998.0    8.0   1.0      -1.403421   
2      24.998001  48.118940    1.0  2020  998.0    8.0   1.0       1.126425   
3      24.998001  48.118940    1.0  2020  998.0    8.0   1.0       1.126425   
4      24.997999  48.118927    1.0  2020  998.0    8.0   1.0       0.410056   
...          ...        ...    ...   ...    ...    ...   ...            ...   
35083  24.937631  47.708638    1.0  2020  998.0    8.0   2.0       0.442170   
35084  24.937625  47.708593    1.0  2020  998.0    8.0   2.0       1.076813   
35085  24.937625  47.708593    1.0  2020  998.0    8.0   2.0       1.076813   
35

In [22]:
def get_unique_dates(track, rgt):
    sql = f"""
        SELECT DISTINCT DATE(time) as date_only
        FROM 'data/tracks_3857_1.parquet'
        WHERE track = {track} AND rgt = {rgt}
        ORDER BY date_only
    """
    return duckdb.query(sql).df()['date_only'].tolist()

In [23]:
get_unique_dates(1, 998)

[Timestamp('2018-12-02 00:00:00'),
 Timestamp('2019-03-03 00:00:00'),
 Timestamp('2020-08-29 00:00:00'),
 Timestamp('2021-08-27 00:00:00'),
 Timestamp('2021-11-26 00:00:00'),
 Timestamp('2023-02-23 00:00:00'),
 Timestamp('2024-02-22 00:00:00'),
 Timestamp('2024-08-21 00:00:00')]

In [1]:
# save_precomputed_dashboard_data.py
import duckdb
import pandas as pd
from utils.db import DuckDBData

db = DuckDBData("data/tracks_3857_1.parquet")

dem_list = [
    "alos_dem", "aster_dem", "copernicus_dem", "fab_dem",
    "nasa_dem", "srtm_dem", "tan_dem"
]

con = duckdb.connect()

# Встанови бажаний DEM, схил, hand_range тощо
dem = "alos_dem"
slope_range = [0, 60]
hand_range = [0, 20]
lulc = None
landform = None

sample_df = db.get_filtered_sample(
    con, dem,
    slope_range=slope_range, hand_range=hand_range,
    lulc=lulc, landform=landform,
    sample_n=20_000
)

filtered_stats = []
for d in dem_list:
    s = db.get_filtered_stats(con, d, slope_range, hand_range, lulc, landform)
    if s:
        s["DEM"] = d
        filtered_stats.append(s)

# Приводимо DEM до красивого формату
for s in filtered_stats:
    s["DEM"] = s["DEM"].replace("_", " ").upper()

# Зберігаємо дві таблиці у файл
df_stats = pd.DataFrame(filtered_stats)
sample_df.to_parquet("data/initial_sample.parquet", index=False)
df_stats.to_parquet("data/initial_stats.parquet", index=False)

con.close()
print("✔ Precomputed dashboard data saved.")




✔ Precomputed dashboard data saved.


In [2]:
import pandas as pd
import duckdb
from utils.db import DuckDBData

db = DuckDBData("data/tracks_3857_1.parquet")
dem_list = [
    "alos_dem", "aster_dem", "copernicus_dem", "fab_dem",
    "nasa_dem", "srtm_dem", "tan_dem"
]

with duckdb.connect() as con:
    stats_all = []
    for d in dem_list:
        s = db.get_dem_stats_sql(con, d, hand_range=None)
        if s:
            s["DEM"] = d.replace("_", " ").upper()
            stats_all.append(s)

df_stats_all = pd.DataFrame(stats_all)
df_stats_all.to_parquet("data/stats_all_cached.parquet", index=False)


In [ ]:
import duckdb
parquet_file = "data/icesat2_dem_filtered_fixed_1.parquet"
dem = "copernicus_dem"

# Унікальні LULC класи
print(duckdb.sql(f"SELECT DISTINCT lulc_name FROM '{parquet_file}' WHERE delta_{dem} IS NOT NULL").df())

# Унікальні landform класи
print(duckdb.sql(f"SELECT DISTINCT {dem}_landform FROM '{parquet_file}' WHERE {dem}_landform IS NOT NULL").df())


In [ ]:
import duckdb
parquet_file = "data/icesat2_dem_filtered_fixed_1.parquet"
duckdb.sql(f"PRAGMA table_info('{parquet_file}')").df()


In [ ]:
dem = "alos_dem"
sql = f"SELECT COUNT(*) as n FROM '{parquet_file}' WHERE delta_{dem} IS NOT NULL"
print(duckdb.sql(sql).df())


In [ ]:
sql = f"SELECT COUNT(*) as n FROM '{parquet_file}' WHERE {dem}_slope IS NOT NULL"
print(duckdb.sql(sql).df())


In [ ]:
sql = f"SELECT DISTINCT delta_{dem} FROM '{parquet_file}' LIMIT 10"
print(duckdb.sql(sql).df())

In [ ]:
import duckdb
parquet_file = "data/icesat2_dem_filtered_fixed_1.parquet"
dem = "alos_dem"
dff = duckdb.sql(f"SELECT delta_{dem} FROM '{parquet_file}' WHERE delta_{dem} IS NOT NULL LIMIT 20").df()
print(dff.head())
print(dff.shape)



In [ ]:
import duckdb
parquet_file = "data/icesat2_dem_filtered_fixed_1.parquet"
dem = "alos_dem"
print("Check if parquet readable and not empty...")
dff = duckdb.sql(f"SELECT delta_{dem} FROM '{parquet_file}' WHERE delta_{dem} IS NOT NULL").df()
print(dff)


In [ ]:
import plotly.graph_objects as go
fig = go.Figure()
fig.add_box(y=dff[f"delta_{dem}"], name=f"delta_{dem}")
fig.show()


In [1]:
from pathlib import Path

# Шлях до каталогу з COG-файлами
base_dir = Path("data/COG")

# Рекурсивно знайдемо всі .tif
for tif_file in base_dir.rglob("*.tif"):
    if "_utm32635" in tif_file.name:
        new_name = tif_file.name.replace("_utm32635", "")
        new_path = tif_file.with_name(new_name)
        tif_file.rename(new_path)
        print(f"✅ Renamed: {tif_file.name} → {new_name}")

✅ Renamed: alos_dem_utm32635_hand_2000_flood_10m_cog.tif → alos_dem_hand_2000_flood_10m_cog.tif
✅ Renamed: alos_dem_utm32635_hand_2000_flood_1m_cog.tif → alos_dem_hand_2000_flood_1m_cog.tif
✅ Renamed: alos_dem_utm32635_hand_2000_flood_2m_cog.tif → alos_dem_hand_2000_flood_2m_cog.tif
✅ Renamed: alos_dem_utm32635_hand_2000_flood_3m_cog.tif → alos_dem_hand_2000_flood_3m_cog.tif
✅ Renamed: alos_dem_utm32635_hand_2000_flood_4m_cog.tif → alos_dem_hand_2000_flood_4m_cog.tif
✅ Renamed: alos_dem_utm32635_hand_2000_flood_5m_cog.tif → alos_dem_hand_2000_flood_5m_cog.tif
✅ Renamed: alos_dem_utm32635_hand_2000_flood_6m_cog.tif → alos_dem_hand_2000_flood_6m_cog.tif
✅ Renamed: alos_dem_utm32635_hand_2000_flood_7m_cog.tif → alos_dem_hand_2000_flood_7m_cog.tif
✅ Renamed: alos_dem_utm32635_hand_2000_flood_8m_cog.tif → alos_dem_hand_2000_flood_8m_cog.tif
✅ Renamed: alos_dem_utm32635_hand_2000_flood_9m_cog.tif → alos_dem_hand_2000_flood_9m_cog.tif
✅ Renamed: aster_dem_utm32635_hand_2000_flood_10m_cog.tif 